In [1]:
from pathlib import Path
import altair as alt
import duckdb
from utils import (
    loads_cae_db,
    executes_query_duckdb
)

In [2]:
cae_path = Path(
    "../../kids_chances/data/raw/cae_history.txt"
)

destionation_db = Path("../data/cae_db.duckdb")
cae = loads_cae_db(cae_path, destionation_db)

In [3]:
query = """
select distinct(tipo_beneficiario) from cae limit 10000"""
executes_query_duckdb(query, cae)

,tipo_beneficiario
0,NUEVO BENEFICIARIO
1,SIN CREDITO POR NO MATRICULA O POR SUSPENSION
2,EGRESO
3,DESERCION
4,BENEFICIARIO RENOVANTE
5,GRATUIDAD


# Big Table with Summary

In [ ]:
query_summary = """

SELECT id_2023 as id,
año_licitacion as año_solicitud,
        genero_registrado as genero,
        dependencia_colegio as colegio,
        region_grupo_familiar as region,
        quintil_ingreso_grupo_familiar as quintil,
        COUNT(CASE WHEN tipo_beneficiario = 'NUEVO BENEFICIARIO' THEN tipo_beneficiario END)
            as total_prestamos,
        COUNT(DISTINCT(nombre_carrera || '-' || nombre_ies)) as cantidad_carreras,
        COUNT(CASE WHEN arancel_solicitado > 0 then arancel_solicitado END)
            as años_financiados,
        COUNT(DISTINCT(CASE WHEN tipo_beneficiario = 'EGRESO' then (tipo_beneficiario || nombre_carrera || nombre_ies) END))
            as total_egresos,
        COUNT(DISTINCT(CASE WHEN tipo_beneficiario = 'DESERCION' then (tipo_beneficiario || nombre_carrera || nombre_ies) END))
            as total_deserciones,
        ROUND(SUM(total_prestado_ajustado),0) as total_prestado,
        ROUND(AVG(porcentaje_financiado),1) as porcentaje_financiado

FROM cae
GROUP BY id, genero, colegio, region, quintil, año_solicitud
"""

In [ ]:
final_table_to_query = executes_query_duckdb(query_summary, cae)

In [6]:
final_table_to_query

,id,año_solicitud,genero,colegio,region,quintil,total_prestamos,cantidad_carreras,años_financiados,total_egresos,total_deserciones,total_prestado,porcentaje_financiado
0,105.111,2009,M,Estudiante de Colegio o Liceo Particular subve...,REGION DE ARICA Y PARINACOTA,quintil 1 y 2,0,1,6,1,0,17279611.0,0.3
1,323.725,2010,M,Particular subvencionado,REGION DEL MAULE,quintil 1 y 2,1,1,5,1,0,17784398.0,1.0
2,324.597,2010,M,Particular pagado,REGION METROPOLITANA,quintil 4,0,1,5,1,0,13231790.0,1.0
3,326.008,2012,F,Municipal,REGION DEL MAULE,quintil 1 y 2,1,3,2,0,1,2821852.0,0.1
4,896.141,2017,M,Municipal,REGION DEL BIO-BIO,quintil 1 y 2,1,1,4,1,0,13944041.0,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1290988,1.192.886,2024,F,Municipal DAEM,REGION DE COQUIMBO,quintil 4,1,1,1,0,0,3051602.0,1.0
1290989,1.224.414,2024,F,Particular Subvencionado,REGION DE TARAPACA,quintil 5,1,1,1,0,0,5983254.0,1.0
1290990,283.218,2009,M,Particular subvencionado,REGION METROPOLITANA,quintil 1 y 2,1,1,1,0,0,3865582.0,1.0
1290991,27.513,2009,F,Municipal,REGION METROPOLITANA,quintil 3,1,1,1,0,0,1863273.0,1.0


---

---

# Aggregated Total Borrowed

In [13]:
DOLAR = 950 # 1 dollar is $950 CLP

query_total_borrowed = f"""
Select 
    region_grupo_familiar as region,
    quintil_ingreso_grupo_familiar as quintil,
    genero_registrado as genero,
    año_operacion as año_financiado,
    round(SUM(total_prestado_ajustado)/{DOLAR}, 0) as total_prestado
FROM cae
GROUP BY region, quintil, genero, año_financiado
"""

total_borrowed = executes_query_duckdb(query_total_borrowed, cae)
total_borrowed

,region,quintil,genero,año_financiado,total_prestado
0,REGION DE ÑUBLE,quintil 1 y 2,M,2016,2070239.0
1,REGION DE ARICA Y PARINACOTA,quintil 4,M,2017,570400.0
2,REGION DE VALPARAISO,quintil 4,F,2015,9985924.0
3,REGION DEL BIO-BIO,quintil 1 y 2,F,2015,39557695.0
4,REGION DE LOS RIOS,quintil 1 y 2,F,2015,7047576.0
...,...,...,...,...,...
2427,REGION DE ÑUBLE,quintil 5,M,2011,89166.0
2428,REGION DE AYSEN DEL G. C. IBAÑEZ,quintil 3,M,2008,67784.0
2429,REGION DE TARAPACA,quintil 5,M,2010,18320.0
2430,REGION DE ATACAMA,quintil 3,F,2006,18112.0


In [14]:
total_borrowed.to_json("../data/total_borrowed.json",
                       orient="records",
                       lines=False,
                       force_ascii=False,
                       indent=4)

---

---

In [4]:
h = executes_query_duckdb("select * from cae limit 100000", cae)
h

,id_2023,año_licitacion,año_operacion,genero_registrado,nacionalidad,region_grupo_familiar,quintil_ingreso_grupo_familiar,cambio_ies_carrera,tipo_beneficiario,nivel_estudio_al_ingreso,...,duracion_formal_carrera,contador_años_financiados,dependencia_colegio,arancel_solicitado,arancel_referencia,tipo_titulo,jornada,banco_administrador,porcentaje_financiado,total_prestado_ajustado
0,667.946,2012,2012,F,Chilena,REGION DE VALPARAISO,quintil 4,NO,NUEVO BENEFICIARIO,PRIMER AÑO,...,5,1,Particular subvencionado,2043680,2043680,Profesional con Licenciatura,Diurna,ESTADO,1.000000,3449731.840
1,958.754,2017,2017,F,Chilena,REGION DE LA ARAUCANIA,quintil 5,NO,NUEVO BENEFICIARIO,PRIMER AÑO,...,7,1,Particular subvencionado,3674581,3674582,Profesional con Licenciatura,Diurna,ESTADO,1.000000,5250976.249
2,668.482,2012,2012,M,Chilena,REGION METROPOLITANA,quintil 5,NO,NUEVO BENEFICIARIO,PRIMER AÑO,...,4,1,Particular subvencionado,820429,2109075,Profesional con Licenciatura,Diurna,ESTADO,0.388999,1384884.152
3,676.859,2012,2012,M,Chilena,REGION METROPOLITANA,quintil 1 y 2,NO,NUEVO BENEFICIARIO,PRIMER AÑO,...,3,1,Municipal,1113000,1113000,Técnico,Diurna,ESTADO,1.000000,1878744.000
4,677.231,2012,2012,F,Chilena,REGION METROPOLITANA,quintil 1 y 2,NO,NUEVO BENEFICIARIO,PRIMER AÑO,...,5,1,Particular subvencionado,1523626,1523626,Profesional,Diurna,ESTADO,1.000000,2571880.688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,398.205,2010,2010,F,Chilena,REGION DEL MAULE,quintil 1 y 2,NO,NUEVO BENEFICIARIO,PRIMER AÑO,...,5,1,Municipal,1340000,1340000,Profesional con Licenciatura,Diurna,SCOTIABANK,1.000000,2398600.000
99996,283.103,2008,2008,M,Chilena,REGION DE VALPARAISO,quintil 1 y 2,NO,NUEVO BENEFICIARIO,PRIMER AÑO,...,4,1,Estudiante de Colegio o Liceo Particular subve...,1259168,1259168,Profesional,Diurna,SCOTIABANK,1.000000,2287908.256
99997,543.917,2011,2017,M,Chilena,REGION METROPOLITANA,quintil 1 y 2,NO,BENEFICIARIO RENOVANTE,PRIMER AÑO,...,5,6,Municipal,0,1917892,Profesional con Licenciatura,Diurna,ITAU,0.000000,0.000
99998,143.034,2008,2010,F,Chilena,REGION DE COQUIMBO,quintil 1 y 2,NO,BENEFICIARIO RENOVANTE,PRIMER AÑO,...,3,3,Estudiante de Colegio o Liceo Fiscal o Municipal,490000,990000,Técnico,Diurna,ESTADO,0.494949,877100.000


In [ ]:
final_table_to_query.to_parquet("../data/summary_table.parquet")